<a href="https://colab.research.google.com/github/prithvijaunjale/Deep-Learning/blob/master/pytorch_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [ ]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from pprint import pprint 

from sklearn.metrics import accuracy_score, f1_score, classification_report

### Data Preprocessing

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])

In [ ]:
train_data = datasets.CIFAR10('CIFAR10_data/train/', train=True, download=True, transform=transform)
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

test_data = datasets.CIFAR10('CIFAR10_data/test/', train=False, download=True, transform=transform)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

Extracting CIFAR10_data/train/cifar-10-python.tar.gz to CIFAR10_data/train/


Extracting CIFAR10_data/test/cifar-10-python.tar.gz to CIFAR10_data/test/


In [ ]:
b_images, b_labels = next(iter(train_dataloader))
print('1 batch:', b_images.shape)
print('Flattened:', b_images.view(b_images.shape[0], -1).shape)
print('Labels:', b_labels.shape)

1 batch: torch.Size([64, 3, 32, 32])
Flattened: torch.Size([64, 3072])
Labels: torch.Size([64])


In [ ]:
b_labels

tensor([4, 0, 9, 3, 6, 1, 7, 6, 6, 5, 3, 8, 3, 6, 2, 7, 0, 0, 7, 2, 7, 7, 0, 7,
        2, 9, 3, 8, 1, 5, 6, 9, 5, 7, 5, 9, 8, 0, 5, 0, 6, 5, 0, 3, 0, 8, 4, 1,
        7, 1, 2, 6, 7, 2, 7, 1, 9, 5, 5, 6, 1, 0, 9, 6])

### Train Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# define model architecure
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        # input dimensions: 32x32x3
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)

        # (Input | 32*32) -> (MaxPool | 16*16) -> (MaxPool | 8*8) -> (MaxPool | 4*4)
        self.fc1 = nn.Linear(in_features=64*4*4, out_features=512)
        self.output = nn.Linear(in_features=512, out_features=10)

        self.dropout = nn.Dropout(p=0.25)

        self.maxpool = nn.MaxPool2d((2, 2))

    def forward(self, x):
        conv1_out = F.relu(self.conv1(x))
        x = self.maxpool(conv1_out)
        conv2_out = F.relu(self.conv2(x))
        x = self.maxpool(conv2_out)
        conv3_out = F.relu(self.conv3(x))
        final_pool_out = self.maxpool(conv3_out)

        # flatten
        x = final_pool_out.view(-1, 64*4*4)
        # (Input | 32*32) -> (MaxPool | 16*16) -> (MaxPool | 8*8) -> (MaxPool | 4*4)

        x = self.dropout(x)
        x = F.relu(self.fc1(x))

        x = self.dropout(x)
        x = F.relu(self.output(x))

        return x, conv1_out, conv2_out, conv3_out, final_pool_out

In [ ]:
model = Model()
model.to(device)

Model(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (output): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (maxpool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)
epochs = 5

In [ ]:
training_losses, val_losses = [], []
conv_outs = []
CONV_OUT = False

for epoch in range(epochs):
    # Training
    train_loss = 0
    model.train()
    steps = 0
    for batch in tqdm(train_dataloader, desc='Epoch ' + str(epoch)):
        b_inputs, b_labels = batch
        b_inputs, b_labels = b_inputs.to(device), b_labels.to(device)

        # clear accumulated gradients
        optimizer.zero_grad()

        # forward pass
        logits = model.forward(b_inputs)
        print(logits[0].shape, b_labels.shape)
        if CONV_OUT:
            conv_outs.append(logits[1:])
            if steps == 1:
                break

        # calc loss
        loss = criterion(logits[0], b_labels)
        train_loss += loss.item()

        # backward pass
        loss.backward()

        # update weights
        optimizer.step()
        steps += 1
    
    training_losses.append(train_loss/len(train_dataloader))
    print('Avg training loss:', train_loss/len(train_dataloader))

    # Validation
    val_loss = 0
    pred, truth = [], []
    model.eval()
    for batch in test_dataloader:
        b_inputs, b_labels = batch
        b_inputs, b_labels = b_inputs.to(device), b_labels.to(device)

        with torch.no_grad():
            logits = model.forward(b_inputs)
            loss = criterion(logits[0], b_labels)

            val_loss += loss.item()

            logits = logits[0].detach().cpu().numpy()
            b_labels = b_labels.detach().cpu().numpy()

            pred.extend(np.argmax(logits, axis=1))
            truth.extend(b_labels)

    val_losses.append(val_loss/len(test_dataloader))
    print('Validation loss:', val_loss/len(test_dataloader))
    print('Validation accuracy:', accuracy_score(truth, pred))

### Understanding Input & Filter dimensions

In [ ]:
state_dict = model.state_dict()
pprint(state_dict.keys())

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'conv3.weight', 'conv3.bias', 'fc1.weight', 'fc1.bias', 'output.weight', 'output.bias'])


In [ ]:
# input
b_images, b_labels = next(iter(train_dataloader))
b_images.shape

# BCHW
# 64 - batch size
# 3 - no. of channels
# 32 - height of input
# 32 - width of input

torch.Size([64, 3, 32, 32])

In [ ]:
# conv2d_1
state_dict['conv1.weight'].shape

# filter
# 16 - no. of filters
# 3 - depth (no. of input channels for the first conv layer. Otherwise, no. of filters of the prev conv layer)
# 3 - height of filter
# 3 - width of filter

torch.Size([16, 3, 3, 3])

In [ ]:
# conv2d_2
state_dict['conv2.weight'].shape

torch.Size([32, 16, 3, 3])

In [ ]:
# conv2d_3
state_dict['conv3.weight'].shape

torch.Size([64, 32, 3, 3])

### Understanding Conv2d Output dimensions

**Input:** n x n = 32 x 32 <br>
**Filter:** f x f = 3 x 3 <br>

**Feature Map:** <br>
* <u>Without Padding</u><br>
= (n – f + 1) x (n – f + 1) <br>
= (32 - 3 + 1) x (32 - 3 + 1) <br>
= 30 x 30 <br>

* <u>With Padding</u><br>
= (n + 2p – f + 1) x (n + 2p – f + 1) <br>
= (32 + 2 x 1 - 3 + 1) x (32 + 2 x 1 - 3 + 1) <br>
= 32 x 32


In [ ]:
conv1_out = conv_outs[0][0]
conv1_out.shape

# 64 - batch size
# 16 - no. of feature maps
# 32 - height of feature map
# 32 - height of feature map

torch.Size([64, 16, 32, 32])

In [ ]:
conv2_out = conv_outs[0][1]
conv2_out.shape

torch.Size([64, 32, 16, 16])

In [ ]:
conv3_out = conv_outs[0][2]
conv3_out.shape

torch.Size([64, 64, 8, 8])

In [ ]:
final_pool_out = conv_outs[0][3]
final_pool_out.shape

torch.Size([64, 64, 4, 4])

### Evaluation

In [ ]:
report = classification_report(truth, pred, digits=4)
print('Clasification report:\n', report)

Clasification report:
               precision    recall  f1-score   support

           0     0.5559    0.7610    0.6425      1000
           1     0.8679    0.7950    0.8299      1000
           2     0.0000    0.0000    0.0000      1000
           3     0.0000    0.0000    0.0000      1000
           4     0.5495    0.6100    0.5782      1000
           5     0.3866    0.7570    0.5118      1000
           6     0.5573    0.8760    0.6812      1000
           7     0.7423    0.6540    0.6954      1000
           8     0.6752    0.8460    0.7510      1000
           9     0.8077    0.7600    0.7831      1000

    accuracy                         0.6059     10000
   macro avg     0.5142    0.6059    0.5473     10000
weighted avg     0.5142    0.6059    0.5473     10000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
plt.plot(training_losses, label='Train loss')
plt.plot(val_losses, label='Val loss')
plt.legend()